In [1]:
# Carga de librerías
import pandas as pd

In [2]:
# Leer archivos CSV
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
sales_train_val = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
sell_prices = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
population_data = pd.read_csv('m5-forecasting-accuracy/population_data.csv')

In [3]:
# Verificar el tamaño de las tablas originales
print("Shape de calendar:", calendar.shape)
print("Shape de sell_prices:", sell_prices.shape)
print("Shape de sales_train_val:", sales_train_val.shape)

# Revisar los primeros registros para cada tabla
print("\nPrimeras filas de calendar:")
print(calendar.head())

print("\nPrimeras filas de sell_prices:")
print(sell_prices.head())

print("\nPrimeras filas de sales_train_val:")
print(sales_train_val.iloc[:, :10].head())

# Revisar los tipos de datos
print("\nTipos de datos de calendar:")
print(calendar.dtypes)

print("\nTipos de datos de sell_prices:")
print(sell_prices.dtypes)

print("\nTipos de datos de sales_train_val:")
print(sales_train_val.dtypes)

# Revisar estadísticas de las columnas numéricas
print("\nEstadísticas de calendar:")
print(calendar.describe(include='all'))

print("\nEstadísticas de sell_prices:")
print(sell_prices.describe(include='all'))

print("\nEstadísticas de sales_train_val:")
print(sales_train_val.describe(include='all'))


Shape de calendar: (1969, 14)
Shape de sell_prices: (6841121, 4)
Shape de sales_train_val: (30490, 1919)

Primeras filas de calendar:
         date  wm_yr_wk    weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101   Saturday     1      1  2011  d_1          NaN   
1  2011-01-30     11101     Sunday     2      1  2011  d_2          NaN   
2  2011-01-31     11101     Monday     3      1  2011  d_3          NaN   
3  2011-02-01     11101    Tuesday     4      2  2011  d_4          NaN   
4  2011-02-02     11101  Wednesday     5      2  2011  d_5          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  
0          NaN          NaN          NaN        0        0        0  
1          NaN          NaN          NaN        0        0        0  
2          NaN          NaN          NaN        0        0        0  
3          NaN          NaN          NaN        1        1        0  
4          NaN          NaN          NaN        1        0       

In [4]:
# Verificar y optimizar valores nulos en 'calendar'
print(calendar.isnull().sum())
calendar.fillna('N/A', inplace=True)

# Convertir día a entero sin el prefijo que tenía y otras columnas a tipo categórico para reducir memoria
calendar['d'] = calendar['d'].str.replace('d_', '').astype(int)
categorical_columns = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for col in categorical_columns:
    calendar[col] = calendar[col].astype('category')

# Revisar tipos de datos optimizados
print(calendar.dtypes)

date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64
date              object
wm_yr_wk           int64
weekday         category
wday               int64
month              int64
year               int64
d                  int32
event_name_1    category
event_type_1    category
event_name_2    category
event_type_2    category
snap_CA            int64
snap_TX            int64
snap_WI            int64
dtype: object


In [5]:
# Verificar valores nulos y estadística de 'sell_prices'
print(sell_prices.isnull().sum())
print(sell_prices['sell_price'].describe())

# Convertir 'wm_yr_wk' a entero y 'sell_price' a float
sell_prices['wm_yr_wk'] = sell_prices['wm_yr_wk'].astype('int16')
sell_prices['sell_price'] = sell_prices['sell_price'].astype('float32')

# Optimizar tipos de datos categóricos
sell_prices['store_id'] = sell_prices['store_id'].astype('category')
sell_prices['item_id'] = sell_prices['item_id'].astype('category')

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64
count    6.841121e+06
mean     4.410952e+00
std      3.408814e+00
min      1.000000e-02
25%      2.180000e+00
50%      3.470000e+00
75%      5.840000e+00
max      1.073200e+02
Name: sell_price, dtype: float64


In [6]:
# Transformar ventas a formato largo con melt para evitar el manejo de tantas columnas
ventas_long = sales_train_val.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                                   var_name='d', 
                                   value_name='sales')

# Convertir día a entero eliminando el prefijo
ventas_long['d'] = ventas_long['d'].str.replace('d_', '').astype(int)

# Optimizar tipos de datos categóricos
for col in ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']:
    ventas_long[col] = ventas_long[col].astype('category')

In [7]:
# Merge de 'ventas_long' y 'calendar'
calendar_cols = ['d', 'wm_yr_wk', 'weekday','wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
ventas_long = pd.merge(ventas_long, calendar[calendar_cols], on='d', how='left')


In [8]:
# Merge de 'ventas_long' con 'sell_prices'
sell_prices_cols = ['store_id', 'item_id', 'wm_yr_wk', 'sell_price']
ventas_completa = pd.merge(ventas_long, sell_prices[sell_prices_cols], on=['store_id', 'item_id', 'wm_yr_wk'], how='left')


In [9]:
# Optimizar 'sell_price' a float32
ventas_completa['sell_price'] = ventas_completa['sell_price'].astype('float32')

# Verificar la forma y las columnas de la tabla resultante
print("ventas_completa:", ventas_completa.shape)
print(ventas_completa.dtypes)

ventas_completa: (58327370, 21)
id                object
item_id         category
dept_id         category
cat_id          category
store_id        category
state_id        category
d                  int32
sales              int64
wm_yr_wk           int64
weekday         category
wday               int64
month              int64
year               int64
event_name_1    category
event_type_1    category
event_name_2    category
event_type_2    category
snap_CA            int64
snap_TX            int64
snap_WI            int64
sell_price       float32
dtype: object


In [10]:
# Optimización de tipos de datos para minimizar el uso de memoria
ventas_completa['sales'] = ventas_completa['sales'].astype('int32')
ventas_completa['wm_yr_wk'] = ventas_completa['wm_yr_wk'].astype('int32')
ventas_completa['month'] = ventas_completa['month'].astype('int8')

In [11]:
# Revisar combinaciones faltantes para verificar si hay ventas sin precio
combinaciones_faltantes = ventas_completa[ventas_completa['sell_price'].isnull()][['store_id', 'item_id', 'wm_yr_wk']].drop_duplicates()
print(combinaciones_faltantes.shape)
print(combinaciones_faltantes.head())

# Filtrar filas con `sell_price` nulo y `sales` mayor a 0
ventas_con_ventas_sin_precio = ventas_completa[(ventas_completa['sales'] > 0) & (ventas_completa['sell_price'].isnull())]
print(ventas_con_ventas_sin_precio.head())

# Contar cuántas filas tienen ventas pero no tienen `sell_price`
print(f"Filas con ventas pero sin precio asociado: {ventas_con_ventas_sin_precio.shape[0]}")


(1757059, 3)
  store_id        item_id  wm_yr_wk
0     CA_1  HOBBIES_1_001     11101
1     CA_1  HOBBIES_1_002     11101
2     CA_1  HOBBIES_1_003     11101
3     CA_1  HOBBIES_1_004     11101
4     CA_1  HOBBIES_1_005     11101
Empty DataFrame
Columns: [id, item_id, dept_id, cat_id, store_id, state_id, d, sales, wm_yr_wk, weekday, wday, month, year, event_name_1, event_type_1, event_name_2, event_type_2, snap_CA, snap_TX, snap_WI, sell_price]
Index: []

[0 rows x 21 columns]
Filas con ventas pero sin precio asociado: 0


In [12]:
# Filtrar filas que tienen sell_price no nulo, ventas mayor a 0 y eliminar '_validation' de los ids
ventas_completa = ventas_completa[(ventas_completa['sell_price'].notnull()) & (ventas_completa['sales'] > 0)]
ventas_completa['id'] = ventas_completa['id'].str.replace('_validation', '')
# Guardar el DataFrame en un archivo CSV
# ventas_completa.to_csv('ventas_completa.csv', index=False, encoding='utf-8')


In [13]:
# Descripción de la tabla resultante
print(ventas_completa.describe())
print(ventas_completa.shape)
print(ventas_completa.dtypes)

                  d         sales      wm_yr_wk          wday         month  \
count  1.855028e+07  1.855028e+07  1.855028e+07  1.855028e+07  1.855028e+07   
mean   1.064629e+03  3.541479e+00  1.136865e+04  3.905839e+00  6.347037e+00   
std    5.385924e+02  6.214025e+00  1.471289e+02  2.033604e+00  3.459276e+00   
min    1.000000e+00  1.000000e+00  1.110100e+04  1.000000e+00  1.000000e+00   
25%    6.240000e+02  1.000000e+00  1.123800e+04  2.000000e+00  3.000000e+00   
50%    1.111000e+03  2.000000e+00  1.140200e+04  4.000000e+00  6.000000e+00   
75%    1.535000e+03  4.000000e+00  1.151100e+04  6.000000e+00  9.000000e+00   
max    1.913000e+03  7.630000e+02  1.161300e+04  7.000000e+00  1.200000e+01   

               year       snap_CA       snap_TX       snap_WI    sell_price  
count  1.855028e+07  1.855028e+07  1.855028e+07  1.855028e+07  1.855028e+07  
mean   2.013505e+03  3.353067e-01  3.368322e-01  3.364867e-01  3.821744e+00  
std    1.502883e+00  4.720976e-01  4.726270e-01  4.725

In [14]:
# Crear la columna de valor de ventas multiplicando las ventas por el precio de venta
ventas_completa['valor_ventas'] = ventas_completa['sales'] * ventas_completa['sell_price']

# Calcular ventas totales por estado y el porcentaje de ventas por estado
ventas_por_estado = ventas_completa.groupby('state_id')['valor_ventas'].sum().reset_index()
total_ventas = ventas_por_estado['valor_ventas'].sum()
ventas_por_estado['porcentaje_ventas'] = (ventas_por_estado['valor_ventas'] / total_ventas) * 100

# Calcular el estado con más ventas y calcular la diferencia relativa en porcentaje con respecto al estado con más ventas
estado_top = ventas_por_estado.loc[ventas_por_estado['valor_ventas'].idxmax()] 
ventas_por_estado['diferencia_relativa'] = ((estado_top['porcentaje_ventas'] - ventas_por_estado['porcentaje_ventas']) / ventas_por_estado['porcentaje_ventas']) * 100

print("Diferencia relativa entre California y el resto de los estados")
print(ventas_por_estado)


Diferencia relativa entre California y el resto de los estados
  state_id  valor_ventas  porcentaje_ventas  diferencia_relativa
0       CA  8.424556e+07          44.888695             0.000000
1       TX  5.404413e+07          28.796417            55.882921
2       WI  4.938688e+07          26.314888            70.582885


In [15]:
# Calcular ventas totales por categoría y el porcentaje de ventas por categoría
ventas_por_categoria = ventas_completa.groupby('cat_id')['sales'].sum().reset_index()
total_ventas = ventas_por_categoria['sales'].sum()
ventas_por_categoria['porcentaje_ventas'] = (ventas_por_categoria['sales'] / total_ventas) * 100
categoria_top = ventas_por_categoria.loc[ventas_por_categoria['sales'].idxmax()]

# Calcular la diferencia relativa en porcentaje con respecto a la categoría con más ventas
ventas_por_categoria['diferencia_relativa'] = ((categoria_top['porcentaje_ventas'] - ventas_por_categoria['porcentaje_ventas']) / ventas_por_categoria['porcentaje_ventas']) * 100
print("Diferencia relativa entre la categoría de alimentos y el resto de las categorías")
print(ventas_por_categoria)


Diferencia relativa entre la categoría de alimentos y el resto de las categorías
      cat_id     sales  porcentaje_ventas  diferencia_relativa
0      FOODS  45089939          68.634840             0.000000
1    HOBBIES   6124800           9.323026           636.186308
2  HOUSEHOLD  14480670          22.042134           211.380199


In [16]:
# Crear la columna de ventas totales multiplicando las ventas por el precio de venta
ventas_completa['total_sales'] = ventas_completa['sell_price'] * ventas_completa['sales']

# Calcular el total de ventas por estado
ventas_por_estado = ventas_completa.groupby('state_id').agg({'total_sales': 'sum'}).reset_index()
ventas_por_estado['num_tiendas'] = ventas_por_estado['state_id'].apply(lambda x: 4 if x == 'CA' else 3)

# Calcular el promedio de ventas por tienda considerando los 5 años 4 meses de datos
ventas_por_estado['avg_sales_per_tienda'] = ventas_por_estado['total_sales'] / (ventas_por_estado['num_tiendas'] * 5.33)
pd.set_option('display.float_format', '{:.2f}'.format)
print("Promedio de ventas por tienda por estado")
print(ventas_por_estado[['state_id', 'avg_sales_per_tienda']])


Promedio de ventas por tienda por estado
  state_id  avg_sales_per_tienda
0       CA            3951480.47
1       TX            3379870.35
2       WI            3088610.35


In [17]:
# Definir un diccionario de mapeo para los estados y sus respectivas columnas SNAP
snap_columns = {
    'CA': 'snap_CA',
    'TX': 'snap_TX',
    'WI': 'snap_WI'
}

resultados_snap = []

# Iterar por cada estado y su respectiva columna SNAP y sumar las ventas realizadas bajo SNAP
for state, snap_column in snap_columns.items():
    ventas_snap_activas = ventas_completa[(ventas_completa['state_id'] == state) & (ventas_completa[snap_column] == 1)]
    ventas_totales_snap = ventas_snap_activas['total_sales'].sum()
    ventas_totales_estado = ventas_completa[ventas_completa['state_id'] == state]['total_sales'].sum()

    # Calcular el porcentaje de ventas SNAP activas con respecto a las ventas totales
    porcentaje_ventas_snap = (ventas_totales_snap / ventas_totales_estado) * 100 if ventas_totales_estado != 0 else 0
    

    resultados_snap.append({
        'state': state,
        'ventas_totales_estado': ventas_totales_estado,
        'ventas_snap_activas': ventas_totales_snap,
        'porcentaje_ventas_snap': porcentaje_ventas_snap
    })

# Crear un DataFrame con los resultados y sumar las ventas totales de todos los estados
df_resultados_snap = pd.DataFrame(resultados_snap)
ventas_totales_global = df_resultados_snap['ventas_totales_estado'].sum()
ventas_snap_global = df_resultados_snap['ventas_snap_activas'].sum()

# Calcular el porcentaje de ventas totales bajo SNAP en comparación con las ventas totales globales
porcentaje_global_snap = (ventas_snap_global / ventas_totales_global) * 100
df_resultados_snap.loc[len(df_resultados_snap)] = ['Total', ventas_totales_global, ventas_snap_global, porcentaje_global_snap]
print("Contraste entre Ventas Totales y Ventas SNAP")
print(df_resultados_snap)

Contraste entre Ventas Totales y Ventas SNAP
   state  ventas_totales_estado  ventas_snap_activas  porcentaje_ventas_snap
0     CA            84245563.70          29046890.01                   34.48
1     TX            54044126.95          19039431.83                   35.23
2     WI            49386879.48          18358220.97                   37.17
3  Total           187676570.13          66444542.81                   35.40


In [18]:
# Calcular la población promedio a lo largo de los años para cada estado
average_population = population_data.mean(numeric_only=True)
print("Promedio de población:\n",average_population.drop('Year'))

# Mapeo de código de estados
state_mapping = {
    'CA': 'California',
    'TX': 'Texas',
    'WI': 'Wisconsin'
}
ventas_completa['state_name'] = ventas_completa['state_id'].map(state_mapping)

# Calcular ventas totales y el gasto per cápita dividiendo las ventas totales por la población promedio
total_sales_per_state = ventas_completa.groupby('state_name')['total_sales'].sum()
per_capita_spending = total_sales_per_state / average_population
print("Cada individuo en promedio gastó:\n",per_capita_spending.drop('Year'))


Promedio de población:
 California   38412469.67
Texas        26828075.33
Wisconsin     5766784.67
dtype: float64
Cada individuo en promedio gastó:
 California   2.19
Texas        2.01
Wisconsin    8.56
dtype: float64


In [19]:
# Agrupar por mes y crear una columna con los valores aplicables de las tres columnas SNAP
snap_por_mes = ventas_completa.groupby(['month'])[['snap_CA', 'snap_TX', 'snap_WI']].sum().reset_index()
snap_por_mes['total_snap'] = snap_por_mes['snap_CA'] + snap_por_mes['snap_TX'] + snap_por_mes['snap_WI']

# Ordenar por la columna 'total_snap' en orden descendente
snap_por_mes = snap_por_mes.sort_values(by='total_snap', ascending=False)
print("Ventas totales por mes bajo SNAP")
print(snap_por_mes[[ 'month', 'total_snap']])


Ventas totales por mes bajo SNAP
    month  total_snap
3       4     1819315
2       3     1794950
1       2     1770132
9      10     1524414
10     11     1512550
11     12     1495438
8       9     1494660
7       8     1477605
6       7     1470779
0       1     1469068
5       6     1466711
4       5     1414661


In [20]:
# Agrupar las ventas totales por año, exceptuando 2016
ventas_anuales = ventas_completa[ventas_completa['year'] < 2016].groupby('year')['total_sales'].sum().reset_index()

# Diferencia porcentual relativa año a año
ventas_anuales['diferencia_porcentual'] = ventas_anuales['total_sales'].pct_change() * 100
print("Diferencia porcentual relativa año a año")
print(ventas_anuales)

Diferencia porcentual relativa año a año
   year  total_sales  diferencia_porcentual
0  2011  23891336.11                    NaN
1  2012  32649200.88                  36.66
2  2013  35923373.36                  10.03
3  2014  37861913.22                   5.40
4  2015  42416456.58                  12.03


In [21]:
# Agrupar ventas totales por mes y ordenar los meses por ventas en orden descendente
ventas_mensuales = ventas_completa.groupby('month')['total_sales'].sum().reset_index()
ventas_mensuales = ventas_mensuales.sort_values(by='total_sales', ascending=False)

# Seleccionar los 3 meses con más ventas y los 3 meses con menos ventas
top_3_meses = ventas_mensuales.head(3)
bottom_3_meses = ventas_mensuales.tail(3)
meses_comparacion = pd.concat([top_3_meses, bottom_3_meses]).reset_index(drop=True)

# Calcular el porcentaje de cambio porcentual relativo entre cada mes y el mes anterior
meses_comparacion['porcentaje_cambio'] = (meses_comparacion['total_sales'].shift(1) - meses_comparacion['total_sales']) / meses_comparacion['total_sales'] * 100
meses_comparacion['porcentaje_cambio'] = meses_comparacion['porcentaje_cambio'].fillna(0)
labels = ['Top'] * 3 + ['Bottom'] * 3
meses_comparacion['Tipo'] = labels
meses_comparacion.columns = ['Mes', 'Ventas Totales', 'Cambio Relativo (%)', 'Tipo']

print("Comparación de ventas y cambios relativos positivos entre top y bottom meses:")
print(meses_comparacion)


Comparación de ventas y cambios relativos positivos entre top y bottom meses:
   Mes  Ventas Totales  Cambio Relativo (%)    Tipo
0    3     18553991.46                 0.00     Top
1    4     17267038.87                 7.45     Top
2    2     17033711.63                 1.37     Top
3    6     14567578.69                16.93  Bottom
4   11     14462801.56                 0.72  Bottom
5    5     14394790.09                 0.47  Bottom


In [22]:
# Agrupar ventas totales por día de la semana
ventas_por_dia_semana = ventas_completa.groupby('wday')['total_sales'].sum().reset_index()

# Ordenar los días de la semana por ventas en orden descendente
ventas_por_dia_semana = ventas_por_dia_semana.sort_values(by='total_sales', ascending=False)

# Calcular el porcentaje de cambio positivo entre cada día consecutivo
ventas_por_dia_semana['porcentaje_cambio'] = (ventas_por_dia_semana['total_sales'].shift(1) - ventas_por_dia_semana['total_sales']) / ventas_por_dia_semana['total_sales'] * 100
ventas_por_dia_semana['porcentaje_cambio'] = ventas_por_dia_semana['porcentaje_cambio'].fillna(0)
ventas_por_dia_semana.columns = ['Día de la Semana', 'Ventas Totales($)', 'Cambio Relativo (%)']
print("Comparación de ventas y cambios relativos positivos entre los días de la semana:")
print(ventas_por_dia_semana)


Comparación de ventas y cambios relativos positivos entre los días de la semana:
   Día de la Semana  Ventas Totales($)  Cambio Relativo (%)
1                 2        32618106.54                 0.00
0                 1        32478936.48                 0.43
6                 7        26509174.05                22.52
2                 3        25734141.43                 3.01
3                 4        23589340.54                 9.09
5                 6        23424763.52                 0.70
4                 5        23322107.57                 0.44


In [23]:
# Filtrar ventas donde event_name_1 o event_name_2 no sean 'N/A' y calcular el total de ventas con eventos
ventas_evento = ventas_completa[(ventas_completa['event_name_1'] != 'N/A') | (ventas_completa['event_name_2'] != 'N/A')]
ventas_totales_evento = ventas_evento['total_sales'].sum()
ventas_totales_global = ventas_completa['total_sales'].sum()

# Calcular el porcentaje de ventas con eventos con respecto al total de ventas
porcentaje_ventas_evento = (ventas_totales_evento / ventas_totales_global) * 100
print(f"Total de ventas con eventos: ${ventas_totales_evento}")
print(f"Total de ventas global: ${ventas_totales_global}")
print(f"Porcentaje de ventas con eventos: {porcentaje_ventas_evento:.2f}%")


Total de ventas con eventos: $14381936.478863187
Total de ventas global: $187676570.12839314
Porcentaje de ventas con eventos: 7.66%


In [24]:
# Agrupar por año y mes para calcular las ventas totales mensuales
ventas_mensuales = ventas_completa.groupby(['year', 'month']).agg({'total_sales': 'sum'}).reset_index()
ventas_previas = ventas_mensuales[ventas_mensuales['year'] < 2016]

# Calcular ventas totales entre enero-abril y mayo-diciembre para cada año de 2011 a 2015
ventas_enero_abril = ventas_previas[(ventas_previas['month'] >= 1) & (ventas_previas['month'] <= 4)]
ventas_mayo_diciembre = ventas_previas[(ventas_previas['month'] >= 5) & (ventas_previas['month'] <= 12)]

ventas_enero_abril_anual = ventas_enero_abril.groupby('year')['total_sales'].sum().reset_index(name='ventas_enero_abril')
ventas_mayo_diciembre_anual = ventas_mayo_diciembre.groupby('year')['total_sales'].sum().reset_index(name='ventas_mayo_diciembre')

# Calcular el factor de cambio promedio entre mayo-diciembre y enero-abril Men el nuevo merge
ventas_anual = pd.merge(ventas_enero_abril_anual, ventas_mayo_diciembre_anual, on='year') 
ventas_anual['factor_mayo_diciembre'] = ventas_anual['ventas_mayo_diciembre'] / ventas_anual['ventas_enero_abril']
factor_promedio = ventas_anual['factor_mayo_diciembre'].mean()

# Obtener las ventas reales de enero a abril y predecir las de mayo a diciembre de 2016
ventas_2016_enero_abril = ventas_mensuales[(ventas_mensuales['year'] == 2016) & (ventas_mensuales['month'] <= 4)]['total_sales'].sum()
prediccion_ventas_mayo_diciembre_2016 = ventas_2016_enero_abril * factor_promedio
prediccion_total_2016 = ventas_2016_enero_abril + prediccion_ventas_mayo_diciembre_2016
print(f"Predicción de ventas totales para 2016: ${prediccion_total_2016}")

Predicción de ventas totales para 2016: $49230542.603765115
